In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'TescoLotus'

corpus = load_corpus('../data/' + file_name + '.txt')
print('Total documents', len(corpus))

Total documents 268


In [3]:
f = open('../data/tokenized/sentencepiece/tokenized_' + file_name + '.txt')
# f = open('../data/tokenized/deepcut/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

### Preprocess Corpus

#### Remove Words

In [4]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 962 words
filter frequent words: 906 words
filter letter words: 847 words
filter stop words: 651 words


In [5]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [6]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [7]:
def get_document_vectors(corpus, value):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if value == 'bow':
        vectors = bow_corpus
    elif value == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        vectors = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(vectors):
        for id, score in doc:
            array[i, id] = score

        if value == 'bow' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [9]:
def eval_cluster(document_vectors, result):
    num_cluster = numpy.unique(result['predicted_label'])[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(document_vectors.iloc[i]))
        corpus_centroid.append(numpy.array(document_vectors.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [18]:
min_samples = 7
eps = 0.20
expand_rate = 0.05
epoch = 15

document_vectors = get_document_vectors(idx_corpus, 'bow')

In [ ]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(document_vectors)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(document_vectors)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(document_vectors, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

#### Iterative New SDC

In [19]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(document_vectors, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

100.03328313821454
[67 23 26  8 22 13 13 22 16  8 11 11 11 10  7]
88.03010917023016
[87 80 23  7 11  9 13 20  7 11]
87.12247378404516
[97 79 22  7  9 13 23  7 11]
87.08072165908341
[98 79 21  7  9 13 23  7 11]
87.64103253844698
[97 79 19  9  9 13 23  7 12]
87.72103927709261
[98 79 18  9  9 13 23  7 12]


#### Grouping

In [20]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
6 4 0.21259117663695096
6 0 0.24492892216238513
[0, 1, 2, 3, 0, 5, 0, 7, 8]


### Result

In [21]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [22]:
# result.to_csv('../data/results/filtered/' + file_name + '.csv', index=False)

# result = pandas.read_csv('../data/results/filtered/' + file_name + '.csv')

print(eval_cluster(document_vectors, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

87.72103927709261
[98 79 18  9  9 13 23  7 12]


In [23]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 5, 7, 8), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
3 คิว เปิดช่องเยอะๆหน่อยค่ะ คนไหนใช้ช่องทางด่วน ผิดกติกา จะยอมช่วยเห็นใจคนทำถูกกติกาด้วยจ้า
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
13 เค้าเตอร์มีเกือบ100ช่อง เปิดให้จ่ายตังไม่ถึง10ช่อง พี่ทำไว้โก้ๆหรอ ต่อคิวจ่ายตังกันไปดิ 10กว่านาที
17 เทสโก้โลตัสสาขาบางกะปิทีวีตัวนี้อยู่บริเวณหน้าร้าน Treephaphan Spa ชั้นลอย(ชั้นเดียวกับไปรษณีย์)ใกล้ประตูออกลานจอดรถวันนั้นผมเดินชนเต็มๆ และผมเชื่อว่าต้องมีลูกค้าท่านอื่นเดินชนทีวีตัวนี้มาแล้วหลายครั้ง มันไม่ควรอยู่ตรงนี้นะครับเพราะนี่คือทางเดิน และมันก็ไม่ได้ชิดเข้ามุมกำแพงเลย เดินเลี้ยวขวาจากทีวีตัวนี้ไปก็ลงไปชั้นG ซึ่งมันเป็นทางหลักที่คนเข็นรถลงจากซุปเปอร์ฯใช้ประจำนะครับ ไม่ใช่ทางลับที่ไม่มีคนเดินผ่าน เพดานชั้นนี้ก็ค่อนข้างต่ำอยู่แล้วแถมเอาทีวีตัวนี้มาติด ต้องคนที่มีความสูง150ลงไปเท่านั้นแหล่ะครับที่จะเดินลอดผ่านได้ (ผมสูง175)ผมเชื่อว่าทางตัวเทสโก้โลตัสยึดมั่นในการบริการที่ดีและอำนวยความสะดวกต่อลูกค้าที่มาใช้บริการ แต่ทีวีตัวนี้ ***ไม่อำนวยความสะดวก*

In [24]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
23 เช็คเอาท์ ... หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะโอกาสหน้าเชิญใหม่ค่ะมีคลับการ์ดมั้ยคะแค่เนี้ยะ ... จะตายมั้ย
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
30 ไม่อยากจะเม้าความสาระพัดของโลตัส ในฐานะที่เคยเป็นพนง.มาก่อน อิอิ
32 ไม่ต้องตัดแต้มเป็นคูปองเงินได้ไหมอ่ะ ไม่เคยได้ใช้เพราะ คูปองมันหายก่อน เอาแบบ พอไปซื้อแล้วแต้มครบค่อยถามว่าตัดแต้มไหมคะ อะไรแบบนี้จะดีกว่า  แล้วยังมีโบชัวมาบอกอีกนะ แต่ม เท่านี้แล้วเป็นตั๋วหนังได้ แล้วเล่นตัดไปแล้วมันเหลือแต้ม150จะทำไรได้
40 โลตัสเอ็กเพรส พนักงานหน้าหงิกมากค่ะ ถามอะไรก็ไม่รู้เรื่อง เลยไม่ค่อยเข้า ล่าสุดเรื่องอีแสตมป์ คุยไปคุยมายังไม่ทันรู้เรื่องเดินหนีจ้าคือไรลูกค้าก็ไม่มี นี่ถ้าไม่อยากไปร่มลายลิลัคคุมะนะ อย่างหวังว่าจะเข้า555
45 มารยาท คำพูดคำจา ไม่ต้องเพราะหรอกคะแค่ไม่ตะคอก ไม่ชักสีหน้าใสเวลาถามสินค้าอยู่ตรงไหนเวลาไม่

In [25]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(index=8, options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=8)

*63 ['กลิ่น', 'ห้าง', 'สา', 'ขาบางซื่อ', 'อาหาร', 'ชื้', 'ฝุ่น', 'สกปรก']
*89 ['กําลัง', 'โลตัส', 'เดิน', 'เข้าไป', 'กลิ่น', 'สด', 'เหม็น', 'คลุ้ง']
*102 ['ไปใช้บริการ', 'ร้าน', 'สกปรก', 'สินค้า', 'ฝุ่น', 'เก', 'พื้น', 'เหนียว', 'ตู้', 'เย็น', 'สกปรก', 'พนง', 'คิดเงิน', 'มากกกกกก']
*162 ['ห้องน้ํา', 'ฉีดตูด']
*178 ['อาหาร', '่อย']
*185 ['ห้องน้ํา', 'สกปรก', 'ทุกสาขา']
*191 ['ห้องน้ํา', 'ควรมี', 'สาย', 'ฉีดตูด', 'นะครับ', 'ห้าง', 'ียว', 'ห้าง']
*199 ['แผนก', 'อาหารสด', 'กลิ่น', 'แรงมากๆ', 'ไม่อยาก', 'เดินผ่าน']
*203 ['กลิ่น']
*208 ['ห้องน้ํา', 'บางสาขา', 'ัง', 'หว', 'ัด', 'สกปรก', 'เข้าไป', 'ไม่ได้']
*233 ['ห้องน้ํา', 'โลตัส', 'คร', 'อิ']
*246 ['ห้องน้ํา', 'แถบ', 'ทุกสาขา']


In [18]:
seed = 0
compare = 0

a = numpy.array(document_vectors.iloc[seed]).reshape(1, -1)
b = numpy.array(document_vectors.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['โลตัส', 'เปอร์', 'มาเก็ต', 'ทุกสาขา', 'พนักงาน', 'เยอะกว่านี้', 'พนักงาน', 'แผนก', 'เครื่อง', 'แคชเชียร์', 'มีหน้าที่', 'ตัวเอง', 'ชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัวเอง', 'เครียด', 'หน้าบึ้ง', 'ตึง', 'ใส่', 'ลูกค้า', 'ทํางาน', 'ไม่มี', 'บริการลูกค้า', 'ดี', 'ลูกค้า', '้อ', 'งเรียน', 'ัก', 'แนะนํา', 'ใจ', 'เด็ก', 'โลตัส', 'เก', '่า']
โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะกว่านี้​ ไม่ควรเรียกพนักงานแผนกอื่นไปลงเครื่องแคชเชียร์​บ่อยจนเกินไป​ ทุกคนต้องมีหน้าที่ที่ตัวเองต้องรับผิดชอบ​ พอช่วยแคชเชียร์เสร็จ​ งานแผนกตัวเองเละก็โดนด่า​ พอโดนด่าก็เกิดความเครียด​ หน้าบึ้งตึงใส่ลูกค้า​ ทำงานไม่มีความสุข​ บริการลูกค้าไม่ดี​ แล้วลูกค้าก็ร้องเรียน​ เพราะรักจึงอยากแนะนำ​ จากใจเด็กโลตัสเก่า​
['โลตัส', 'เปอร์', 'มาเก็ต', 'ทุกสาขา', 'พนักงาน', 'เยอะกว่านี้', 'พนักงาน', 'แผนก', 'เครื่อง', 'แคชเชียร์', 'มีหน้าที่', 'ตัวเอง', 'ชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัวเอง', 'เครียด', 'หน้าบึ้ง', 'ตึง', 'ใส่', 'ลูกค้า', 'ทํางาน', 'ไม่มี', 'บริการลูกค้า', 'ดี', 'ลูกค้า', '้อ', 'งเรียน', 'ัก'